In [177]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import interpolate

In [178]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [179]:
from bender_functions import Bender

# Generate signal

Sampling parameters

In [180]:
samplingfreq = 1000.0
outputfreq = 100000.0

waitbefore = 0.1
waitafter = 1.0

## Movement parameters

In [217]:
freq = 2.2     # Hz
amp = 10       # deg
ncycles = 5

scale = 6       # output teeth divided by input teeth

## Activation parameters

In [218]:
activation_duty = 0.3       # fractions of a cycle
activation_phase = 0.1      # fractions of a cycle

activation_pulse_rate = 30  # Hz
start_cycle = 3

is_activation = False        # set to False for passive tests

In [219]:
actburstdur = activation_duty / freq

# make sure the activation is an even number of pulses
actburstdur = np.floor(actburstdur * activation_pulse_rate * 2) / (
        activation_pulse_rate * 2)

actburstduty = actburstdur * freq

print("Activation burst duration: {:.3} sec".format(actburstdur))
print("True activation duty: {:.3}".format(actburstduty))

Activation burst duration: 0.133 sec
True activation duty: 0.293


Start setting up the output

In [220]:
bender = Bender()

In [221]:
movedur = ncycles / freq
totaldur = waitbefore + movedur + waitafter

t = np.arange(0, totaldur, 1.0/samplingfreq)
t -= waitbefore

tnorm = t * freq

Generate the angle and angular velocity signals

In [222]:
angle = amp * np.sin(2*np.pi * freq * t)
anglevel = 2*np.pi * amp * freq * np.cos(2*np.pi * freq * t)

angle[t < 0] = 0
angle[t > movedur] = 0

anglevel[t < 0] = 0
anglevel[t > movedur] = 0

In [223]:
bender.set_bending_signal(t, angle, anglevel)

In [224]:
actcmd = np.zeros_like(t)
Lonoff = []
Ronoff = []

if is_activation:
    pulsedur = 0.01         # 10ms long pulse to start the S88

    startstim = np.zeros((int(actburstdur * samplingfreq + 1),))
    startstim[:int(pulsedur * samplingfreq)] = 5

    bendphase = tnorm - 0.25

    # list of cycles when we'll have activation
    actcycles = list(range(start_cycle, ncycles))

    # also do one cycle before the bending starts
    actcycles.insert(0, -3)
    # and one after
    actcycles.append(ncycles+1)

    for c in actcycles:
        k = np.argmax(bendphase >= c + activation_phase)
        tstart = t[k]
        tend = tstart + actburstdur

        if any(bendphase >= c + activation_phase):
            Lonoff.append([tstart, tend])
        if any(bendphase >= c + activation_phase + 0.5):
            Ronoff.append(np.array([tstart, tend]) + 0.5 / freq)

        np.place(actcmd, np.logical_and(bendphase >= c + activation_phase,
                                        bendphase < c + activation_phase + actburstduty),
                                        startstim)

Lonoff = np.array(Lonoff)
Ronoff = np.array(Ronoff)

In [225]:
bender.set_activation(actcmd)

and plot them:

In [226]:
fig = make_subplots(rows = 2, cols = 1,
                   shared_xaxes=True)
fig.add_trace(
    go.Scatter(x = t, y = angle, mode="lines", name="angle"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(x = t, y = actcmd, mode="lines", name="act"),
    row=1, col=1)

for onoff in Lonoff:
    fig.add_vrect(x0 = onoff[0], x1=onoff[1], fillcolor="black", opacity=0.25, line_width=0,
                      row=1, col=1)

for onoff in Ronoff:
    fig.add_vrect(x0 = onoff[0], x1=onoff[1], opacity=0.7, line_width=1,
                      row=1, col=1)

fig.update_yaxes(title_text = "angle (deg)", row=1)
fig.add_trace(
    go.Scatter(x = t, y = anglevel, mode="lines", name="anglevel"),
    row=2, col=1)

fig.update_yaxes(title_text = "angular velocity (deg/s)", row=2)
fig.update_xaxes(title_text = "time (s)", row=2)

Generate the motor step and direction pulses. 

In [227]:
tout, dig, step, direction = bender.make_motor_stepper_pulses(outputfreq,
                        scale=scale,
                        stepsperrev=1600)

Use the cell below to debug the step and direction pulses, but don't render it every time. It takes a long time to plot the traces, because the output sampling rate is high.

In [228]:
# fig = make_subplots()
# fig.add_trace(go.Scatter(x=tout, y=step, mode="lines", name="step"))
# fig.add_trace(go.Scatter(x=tout, y=direction, mode="lines", name="dir"))
# fig.add_trace(go.Scatter(x=tout, y=dig, mode="lines", name="port"))

# Do data acquisition

In [229]:
device_name = '/Dev1'

## Analog output channel

Sends the pulses to the S88 for muscle activation

In [230]:
bender.set_activation_channel('ao0')

## Digital output channel

Controls the motor

In [231]:
bender.set_motor_channel('port0')

## Analog input channels

Six channels from the force transducer, plus the monitor channel from the S88 stimulator.

In [232]:
SG1_chan = 'ai0'
SG2_chan = 'ai1'
SG3_chan = 'ai2'
SG4_chan = 'ai3'
SG5_chan = 'ai4'
SG6_chan = 'ai5'

In [233]:
activation_monitor_chan = 'ai6'

In [234]:
inchannels = [SG1_chan, SG2_chan, SG3_chan, SG4_chan, SG5_chan, SG6_chan,
                activation_monitor_chan]
inchannel_names = ['SG1', 'SG2', 'SG3', 'SG4', 'SG5', 'SG6',
                    'activation_monitor']

bender.set_input_channels(inchannels, inchannel_names)

Force transducer calibration file

In [235]:
bender.loadCalibration('FT17161.cal')
bender.calibration

array([[ 9.609000e-02, -2.877000e-01,  1.039313e+01, -2.850000e-03,
        -1.706000e-01, -2.490000e-03],
       [ 5.423000e-02, -7.349830e+00,  2.909400e-01, -4.012000e-02,
        -4.740000e-03, -8.726000e-02],
       [-7.387000e-02,  8.691000e-02,  1.065666e+01,  1.484400e-01,
         8.860000e-02,  0.000000e+00],
       [ 6.261130e+00,  3.686710e+00, -4.922800e-01,  1.253000e-02,
        -3.947000e-02, -8.481000e-02],
       [-7.690000e-03, -6.688000e-02,  1.045477e+01, -1.517300e-01,
         8.331000e-02,  1.500000e-04],
       [-6.235270e+00,  3.543590e+00, -3.817600e-01,  2.526000e-02,
         3.202000e-02, -8.559000e-02]])

## Encoder angle input

In [236]:
bender.set_encoder_channel('ctr0', counts_per_rev=10000)

## Output file

In [214]:
outputfile = 'trial_001.h5'
outputfile = bender.increment_file_name(outputfile)

print('Actual output file: {}'.format(outputfile))

Actual output file: trial_001.h5


## Main code block

Sets up the DAQ, sends the output, records the input, and writes it to the file.

In [215]:
aidata = bender.run(device_name)

In [216]:
fig = make_subplots()
fig.add_trace(go.Scatter(x = t, y = bender.angledata, mode="lines"))

In [95]:
forcetorque = bender.applyCalibration(aidata)

In [105]:
fig = make_subplots(rows = 2, cols = 1,
                   shared_xaxes=True)

fig.add_trace(
    go.Scatter(x = t, y = forcetorque[0, :],mode="lines", name='Fx'),
    row=1, col=1)
fig.add_trace(
    go.Scatter(x = t, y = forcetorque[0, :],mode="lines", name='Fx'),
    row=1, col=1)

fig.add_trace(
    go.Scatter(x = t, y = forcetorque[3, :],mode="lines", name='Tx'),
    row=2, col=1)
